# Compute (DLC) Pose Estimation accuracy
First, we'll import the hand labeled ground truth files.

This notebook is designed to work with the standard folder structure found in [DeepLabCut](https://github.com/DeepLabCut/DeepLabCut) projects. We are going to be using our own custom functions for evaluation here, so we're going to use DLC-Live so we can point to individual images rather than entire datasets with a subsection of labels.

In [1]:
from dlclive import DLCLive, Processor
import cv2
from tqdm.notebook import tqdm, trange

"""
To install tqdm (to display progress bars in jupyter) run the following commands in your environment:

1. conda install -c conda-forge tqdm   # conda
2. pip install ipywidgets
3. jupyter nbextension enable --py widgetsnbextension

"""

dlc_proc = Processor()
dlc_trained_model_path = "C:/Users/Legos/Documents/PhD/Blender/OmniTrax/OmniTrax_WIP/DLC/DLC_multi_stick_v1_resnet_50_iteration-0_shuffle-1/"
dlc_live = DLCLive(dlc_trained_model_path, processor=dlc_proc)

now let's load all the data we want to evaluate!

This script needs to point at the location of the "labeled-data" folder of your DLC project, expecting the hierachy:

***labeled-data/VIDEOS/CollectedData_USER.h5***

In [2]:
import os
from pathlib import Path
import pandas as pd
labeled_data_path = Path("C:/Users/Legos/Documents/PhD/Blender/OmniTrax/OmniTrax_WIP/benchmark/STICK_DLC/")

# grab all h5 files
first_h5_found = False
for subdir, dirs, files in os.walk(labeled_data_path):
    for file in files:
        if file[-1] == "5":
            if not first_h5_found:
                df_labeled = pd.read_hdf(os.path.join(subdir, file))
                first_h5_found = True
            else:
                more_labeled = pd.read_hdf(os.path.join(subdir, file))
                df_labeled = df_labeled.append(more_labeled)

df_labeled

scorer                                     Hendrik                           \
bodyparts                                l_antenna                r_antenna   
coords                                           x           y            x   
labeled-data/spec2_cam0_d/img0726.png   140.369533  739.648848   103.072561   
labeled-data/spec2_cam0_d/img0749.png   357.464128  717.504572   347.479778   
labeled-data/spec2_cam0_d/img0750.png   367.380356  711.422910   357.834862   
labeled-data/spec2_cam0_d/img0780.png   805.068688  459.579159   879.173121   
labeled-data/spec2_cam0_d/img0782.png   852.903569  457.464509   922.602618   
...                                            ...         ...          ...   
labeled-data/spec7_cam4_i/img476.png   1737.339885  203.596400  1824.966020   
labeled-data/spec7_cam4_i/img490.png   1873.180917  363.783456  1935.135220   
labeled-data/spec7_cam4_i/img525.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img559.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img647.png           NaN         NaN          NaN   

scorer                                                                      \
bodyparts                                                 head               
coords                                          y            x           y   
labeled-data/spec2_cam0_d/img0726.png  959.479519          NaN         NaN   
labeled-data/spec2_cam0_d/img0749.png  947.031210   154.820928  822.031798   
labeled-data/spec2_cam0_d/img0750.png  936.524933   164.366369  813.146659   
labeled-data/spec2_cam0_d/img0780.png  744.778930   681.627438  649.355238   
labeled-data/spec2_cam0_d/img0782.png  726.569617   711.426452  637.981903   
...                                           ...          ...         ...   
labeled-data/spec7_cam4_i/img476.png   263.015207  1665.850587  343.625668   
labeled-data/spec7_cam4_i/img490.png   455.638521  1771.812008  353.201384   
labeled-data/spec7_cam4_i/img525.png          NaN  2035.843680  349.747242   
labeled-data/spec7_cam4_i/img559.png          NaN          NaN         NaN   
labeled-data/spec7_cam4_i/img647.png          NaN          NaN         NaN   

scorer                                                                       \
bodyparts                                       t1                       t2   
coords                                           x           y            x   
labeled-data/spec2_cam0_d/img0726.png          NaN         NaN          NaN   
labeled-data/spec2_cam0_d/img0749.png   111.885920  820.521995     3.085781   
labeled-data/spec2_cam0_d/img0750.png   119.152956  811.755478    12.379743   
labeled-data/spec2_cam0_d/img0780.png   636.772446  660.319792   537.343880   
labeled-data/spec2_cam0_d/img0782.png   670.367328  647.700039   564.196694   
...                                            ...         ...          ...   
labeled-data/spec7_cam4_i/img476.png   1623.356454  354.184816  1542.746371   
labeled-data/spec7_cam4_i/img490.png   1726.738306  358.002133  1654.193648   
labeled-data/spec7_cam4_i/img525.png   1988.705659  352.857873  1912.854122   
labeled-data/spec7_cam4_i/img559.png           NaN         NaN          NaN   
labeled-data/spec7_cam4_i/img647.png           NaN         NaN          NaN   

scorer                                             ...                 \
bodyparts                                          ... h_r_trochanter   
coords                                          y  ...              x   
labeled-data/spec2_cam0_d/img0726.png         NaN  ...            NaN   
labeled-data/spec2_cam0_d/img0749.png  802.121278  ...            NaN   
labeled-data/spec2_cam0_d/img0750.png  790.887749  ...            NaN   
labeled-data/spec2_cam0_d/img0780.png  668.543207  ...     454.503508   
labeled-data/spec2_cam0_d/img0782.png  655.474548  ...     489.545284   
...                                           ...  ...            ...   
labeled-data/spec7_cam4_i/img476.png

In [3]:
# let's check the labels of the imported annotated data
import numpy as np
point_size = 5

for item in df_labeled.iterrows():
    img_name = item[0]
    img_path = os.path.join(labeled_data_path,img_name)
    img = cv2.imread(str(img_path))

    for coord in range(int((len(item[1])) / 2)):
        if not np.isnan(item[1][coord*2]) and not np.isnan(item[1][coord*2 + 1]):
            img = cv2.circle(img, (int(item[1][coord*2]), int(item[1][coord*2 + 1])),
                                       point_size,
                                       (int(255 * coord / 49), int(255 - 255 * coord / 49), 200), -1)
                   
    cv2.imshow("DLC Labeled IMG",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

**Initialise** the network with the desired input shape. Afterwards the network can be used for inference on the fly

In [4]:
%%capture
desired_dims = (1024,768)
desired_dims = (512,384)

dlc_test_image_path = img_path

dlc_test_image = cv2.imread(dlc_test_image_path)
dlc_test_image_resized = cv2.resize(dlc_test_image,desired_dims)

dlc_live.init_inference(dlc_test_image_resized)

Cool, now let's check how well our trained network performs.

If desired, add a lookup-table to only evaluate performance on specific landmarks. This can be useful when there is a discrepancy between which landmarks have been labeled in the real vs synthetic datasets.

In [ ]:
thresh = 0.1
point_size = 2

all_poses = []
show_outputs = True
print("Running inference...")

for item in tqdm(df_labeled.iterrows(),total=len(df_labeled.index)):
    img_name = item[0]
    img_path = os.path.join(labeled_data_path,img_name)
    
    dlc_image = cv2.imread(img_path)
    dlc_image_resized = cv2.resize(dlc_image,desired_dims)

    pose = dlc_live.get_pose(dlc_image_resized)
    
    if show_outputs:
        for p, point in enumerate(pose):
            if point[2] >= thresh:
                dlc_test_image_resized = cv2.circle(dlc_image_resized, (int(point[0]), int(point[1])),
                                           point_size,
                                           (int(255 * p / 49), int(255 - 255 * p / 49), 200),
                                           -1)

        cv2.imshow("DLC Predicted IMG",dlc_image_resized)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            show_outputs = False
            cv2.destroyAllWindows()
        
    all_poses.append(pose)
    
cv2.destroyAllWindows()

  0%|          | 0/862 [00:00<?, ?it/s]